In [1]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 133kB/s  eta 0:00:01
     |████████████████████████████████| 204kB 24.7MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:

# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-06-26 05:45:28--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-06-26 05:45:28--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-06-26 05:45:29--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Savin

In [5]:
df.show()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [7]:
#training testing
splits = df.randomSplit([0.8,0.2])
df_train = splits[0]
df_test = splits[1]

In [8]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler,Normalizer
from pyspark.ml.linalg import Vectors

In [10]:
indexer = StringIndexer(inputCol="class",outputCol="label")
encoder = OneHotEncoder(inputCol="label", outputCol="labelVec")
vA = VectorAssembler(inputCols=["x","y","z"],outputCol="features")
normalizer = Normalizer(inputCol="features",outputCol="features_norm",p=1.0)

In [11]:
from pyspark.ml.classification import LinearSVC

In [13]:
svc = LinearSVC(maxIter=10,regParam=0.1)

In [16]:
df.createOrReplaceTempView('df')
df_class = spark.sql("select * from df where class in ('Use_telephone','Standup_chair')")


In [19]:
#for binary classification
splits = df_class.randomSplit([0.8,0.2])
df_train = splits[0]
df_test = splits[1]

In [17]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer,encoder,vA,normalizer,svc])

In [20]:
#training
model = pipeline.fit(df_train)

In [21]:
pred = model.transform(df_train)

In [23]:
#evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction')
evaluator.evaluate(pred)

0.9387918505500965

In [24]:
#for testing
pred_test = model.transform(df_test)
evaluator.evaluate(pred_test)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


0.9362888527492641